Serveur chez OVH :
- Mongo DB
- Docker
- Appli de rajout d'élévation

Réunion du 27/10/2022

- Acces MongoDB via MongoDB Compass
- Interpolation : se donner une tolérance. Si le pointage au passage d'un arrêt est trop éloigné des coordonénes gps rajouter un enregistrement par interpolation
- Sauvegarder la donnée nettoyée sous forme brute dans la base MongoDB (collection)
- Dupliquer les lignes avec des destinations finales différentes


In [1]:
import pandas as pd
import glob
from datetime import date, datetime

from pymongo import MongoClient

In [2]:
# Provide the mongodb atlas url to connect python to mongodb using pymongo
def get_database(uri="mongodb://localhost:27017/", database = "Optymo"):
    # Create a connection
    mgClient = MongoClient(uri)
    return mgClient[database]

# provide the collection in the mongodb database
def get_collection(uri="mongodb://localhost:27017/", database = "Optymo", collection = "Courses"):
   # Create a connection
   dbClient = get_database(uri, database)
   # Create the database
   return dbClient[collection]

In [3]:
dbClient = get_database()
for collection in dbClient.list_collection_names():
    print(collection)

Courses


In [4]:
collection = get_collection()
collection.name

'Courses'

In [67]:
## remove all records from the collection (start from fresh data base)
result = collection.delete_many({})

# Read files
Get the file names

In [5]:
filenames = glob.glob("filtered_data_*")
filenames.sort()
filenames

['filtered_data_2019', 'filtered_data_2020']

There is two files (2019 and 2020).

We will loop over theses files in order to extract and register the courses

In [6]:
f=filenames[0]
df = pd.read_csv(f)
df.drop(columns=["Unnamed: 0"], inplace=True)
pd.unique(df["ligne"]), pd.unique(df["state"]), pd.unique(df["finalDestination"])

(array([3, 5, 4, 1, 2]),
 array(['En course', 'En Terminus'], dtype=object),
 array(['Gare TGV', 'Valdoie Mairie', 'Bois Joli', "Prés D'Aumont",
        'Pépinière', 'Offemont', 'Offemont Cerisiers', 'Cerisiers',
        'Liberté Follereau', 'Résidences', 'IUT', 'Châtenois',
        'Belfort Gare', 'Argésians', 'Justice', 'Cravanche', 'Bavilliers',
        'Froideval', 'Gare de Belfort', 'Morvillars Gare', 'Clémenceau',
        'Gare'], dtype=object))

In [10]:
f=filenames[1]
df = pd.read_csv(f)
df.drop(columns=["Unnamed: 0"], inplace=True)
pd.unique(df["ligne"]), pd.unique(df["state"]), pd.unique(df["finalDestination"])

(array([3, 1, 2, 5, 4]),
 array(['En course', 'En Terminus'], dtype=object),
 array(['IUT', 'Châtenois', 'Valdoie Mairie', 'Résidences', 'Justice',
        'Cravanche', 'Bavilliers', 'Liberté Follereau', 'Gare TGV',
        'Belfort Gare', 'Argésians', 'Bois Joli', "Prés D'Aumont",
        'Pépinière', 'Offemont', 'Offemont Cerisiers', 'Cerisiers',
        'Froideval', 'Gare de Belfort', 'Morvillars Gare', 'Clémenceau'],
       dtype=object))

In [11]:
df.columns

Index(['ligne', 'date', 'hour', 'longitude', 'latitude', 'elevation',
       'lastStop', 'finalDestination', 'state', 'Year'],
      dtype='object')

In [12]:
pd.unique(df["ligne"])

array([3, 1, 2, 5, 4])

In [13]:
pd.unique(df["state"])

array(['En course', 'En Terminus'], dtype=object)

In [22]:
pd.unique(df["date"])

array(['02/01/2020', '06/01/2020', '11/01/2020', '12/01/2020',
       '13/01/2020', '15/01/2020', '18/01/2020', '24/01/2020',
       '25/01/2020', '29/01/2020', '01/02/2020', '02/02/2020',
       '05/02/2020', '10/02/2020', '11/02/2020', '12/02/2020',
       '17/02/2020', '18/02/2020', '22/02/2020', '25/02/2020',
       '26/02/2020', '03/03/2020', '04/03/2020', '05/03/2020',
       '08/03/2020', '14/03/2020', '15/03/2020', '16/03/2020',
       '29/06/2020', '12/05/2020', '13/05/2020', '14/05/2020',
       '16/05/2020', '17/05/2020', '18/05/2020', '19/05/2020',
       '20/05/2020', '23/05/2020', '27/05/2020', '28/05/2020',
       '01/07/2020', '04/07/2020', '06/07/2020', '07/07/2020',
       '08/07/2020', '10/07/2020', '14/07/2020', '17/07/2020',
       '18/07/2020', '19/07/2020', '20/07/2020', '23/07/2020',
       '24/07/2020', '26/07/2020', '29/07/2020', '31/07/2020',
       '01/08/2020', '02/08/2020', '06/08/2020', '08/08/2020',
       '11/08/2020', '13/08/2020', '16/08/2020', '17/08

In [29]:
date = df["date"]
date, date[0:2], date[3:5], date[6:10]

('02/01/2020', '02', '01', '2020')

In [43]:
hour = df["hour"][0]
hour, hour[0:2], hour[3:5], hour[6:8]

('05:36:36', '05', '36', '36')

# Register courses

In [7]:
class Extractor:
    def __init__(self, minLenCourses, nCourse, database = "Optymo", collection = "Courses"):
        self.minLenCourses = minLenCourses
        self.nCourse       = nCourse
        self.courses       = get_collection(database = database, collection = collection)
    
    def create_records(self, df, indexes):
        df_rec           = df[['ligne','date','hour', 'longitude', 'latitude', 'elevation','lastStop', 'finalDestination']].iloc[indexes]
        date = df_rec["date"].iloc[0]
        df_rec["Course"]= self.nCourse
        day, month, year  = int(date[0:2]), int(date[3:5]), int(date[6:10])
        for ind in df_rec.index:
            time = df_rec['hour'][ind]
            hour, minute, second = int(time[0:2]), int(time[3:5]), int(time[6:8])
            df_rec["date"][ind]  = datetime(year, month, day, hour, minute, second)
        return df_rec.drop('hour', axis = 1).to_dict(orient='records')
            
    def register_course(self, df, indexes):
        if (len(indexes)) < self.minLenCourses:
            return
        ### look to the fields
        self.nCourse  = self.nCourse+1
        records       = self.create_records(df, indexes)
        insert_result = self.courses.insert_many(records)

        ### does not work
#        _id     = 'Course'+ str(self.nCourse)
#        ligne   = df["ligne"][indexes[0]]
#        date    = str(df["date"][indexes[0]]) # use datetime.datetime ?
#        records = df[['hour', 'longitude', 'latitude', 'elevation','lastStop', 'finalDestination']].iloc[indexes].to_json(orient='records')
#        insert_result = self.courses.insert_one({
#                        "_id" : _id,
#                        "ligne": ligne,
#                        "date": date,
#                        "pointages": records
#        })
        print("Register course number :", self.nCourse)
        return
    
    def extract_courses(self, df):
        """ Extract segment of the bus courses

        Parameters
        ----------
            df : pandas DataFrame
                    input data, the courses of one year
        """
        nRow= df.shape[0]
        if nRow<=1:
            return

        # get initial values
        line  = df['ligne'][0]
        final = df['finalDestination'][0]
        # indexes will contain a bus course
        indexes = []

        ## loop over the rows of the data frame
        for index, row in df.iterrows():
            # remove 'En Terminus' points and initialize indexes array
            if row['state'] == 'En Terminus':
                if len(indexes) != 0:
                    if (c_line==row["ligne"]):
                        indexes.extend([index])
                    self.register_course(df,indexes) # save indexes
                indexes = []       # intialize indexes for the next course
            else:
                # bus is not 'En Terminus' thus start the recording
                c_line    = row["ligne"]
                c_final   = row['finalDestination']

                # build the flags
                is_line_different  = (line  != c_line)
                is_final_different = (final != c_final)

                # Check if the bus change line or change finalDestination (should not exist but...)
                if (is_line_different or is_final_different):
                    final   = c_final
                    line    = c_line
                    self.register_course (df,indexes)  # save indexes find
                    indexes=[index] # start a new index 
                else:
                    indexes.extend([index]) # just add current index
#            if index >10000:
#                break
        return self.nCourse

In [68]:
e = Extractor(100, 0)
e.extract_courses(df)

/tmp/ipykernel_3832846/2760014558.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rec["date"][ind]  = datetime(year, month, day, hour, minute, second)


Register course number : 1
Register course number : 2
Register course number : 3
Register course number : 4
Register course number : 5
Register course number : 6
Register course number : 7
Register course number : 8
Register course number : 9
Register course number : 10
Register course number : 11
Register course number : 12
Register course number : 13
Register course number : 14
Register course number : 15
Register course number : 16
Register course number : 17
Register course number : 18
Register course number : 19
Register course number : 20
Register course number : 21
Register course number : 22
Register course number : 23
Register course number : 24
Register course number : 25
Register course number : 26
Register course number : 27
Register course number : 28
Register course number : 29
Register course number : 30
Register course number : 31
Register course number : 32
Register course number : 33
Register course number : 34
Register course number : 35
Register course number : 36
R

Register course number : 288
Register course number : 289
Register course number : 290
Register course number : 291
Register course number : 292
Register course number : 293
Register course number : 294
Register course number : 295
Register course number : 296
Register course number : 297
Register course number : 298
Register course number : 299
Register course number : 300
Register course number : 301
Register course number : 302
Register course number : 303
Register course number : 304
Register course number : 305
Register course number : 306
Register course number : 307
Register course number : 308
Register course number : 309
Register course number : 310
Register course number : 311
Register course number : 312
Register course number : 313
Register course number : 314
Register course number : 315
Register course number : 316
Register course number : 317
Register course number : 318
Register course number : 319
Register course number : 320
Register course number : 321
Register cours

Register course number : 571
Register course number : 572
Register course number : 573
Register course number : 574
Register course number : 575
Register course number : 576
Register course number : 577
Register course number : 578
Register course number : 579
Register course number : 580
Register course number : 581
Register course number : 582
Register course number : 583
Register course number : 584
Register course number : 585
Register course number : 586
Register course number : 587
Register course number : 588
Register course number : 589
Register course number : 590
Register course number : 591
Register course number : 592
Register course number : 593
Register course number : 594
Register course number : 595
Register course number : 596
Register course number : 597
Register course number : 598
Register course number : 599
Register course number : 600
Register course number : 601
Register course number : 602
Register course number : 603
Register course number : 604
Register cours

Register course number : 854
Register course number : 855
Register course number : 856
Register course number : 857
Register course number : 858
Register course number : 859
Register course number : 860
Register course number : 861
Register course number : 862
Register course number : 863
Register course number : 864
Register course number : 865
Register course number : 866
Register course number : 867
Register course number : 868
Register course number : 869
Register course number : 870
Register course number : 871
Register course number : 872
Register course number : 873
Register course number : 874
Register course number : 875
Register course number : 876
Register course number : 877
Register course number : 878
Register course number : 879
Register course number : 880
Register course number : 881
Register course number : 882
Register course number : 883
Register course number : 884
Register course number : 885
Register course number : 886
Register course number : 887
Register cours

...

Register course number : 3326
Register course number : 3327
Register course number : 3328
Register course number : 3329
Register course number : 3330
Register course number : 3331
Register course number : 3332
Register course number : 3333
Register course number : 3334
Register course number : 3335
Register course number : 3336
Register course number : 3337
Register course number : 3338
Register course number : 3339
Register course number : 3340
Register course number : 3341
Register course number : 3342
Register course number : 3343
Register course number : 3344
Register course number : 3345
Register course number : 3346
Register course number : 3347
Register course number : 3348
Register course number : 3349
Register course number : 3350
Register course number : 3351
Register course number : 3352
Register course number : 3353
Register course number : 3354
Register course number : 3355
Register course number : 3356
Register course number : 3357
Register course number : 3358
Register c

Register course number : 3600
Register course number : 3601
Register course number : 3602
Register course number : 3603
Register course number : 3604
Register course number : 3605
Register course number : 3606
Register course number : 3607
Register course number : 3608
Register course number : 3609
Register course number : 3610
Register course number : 3611
Register course number : 3612
Register course number : 3613
Register course number : 3614
Register course number : 3615
Register course number : 3616
Register course number : 3617
Register course number : 3618
Register course number : 3619
Register course number : 3620
Register course number : 3621
Register course number : 3622
Register course number : 3623
Register course number : 3624
Register course number : 3625
Register course number : 3626
Register course number : 3627
Register course number : 3628
Register course number : 3629
Register course number : 3630
Register course number : 3631
Register course number : 3632
Register c

Register course number : 3874
Register course number : 3875
Register course number : 3876
Register course number : 3877
Register course number : 3878
Register course number : 3879
Register course number : 3880
Register course number : 3881
Register course number : 3882
Register course number : 3883
Register course number : 3884
Register course number : 3885
Register course number : 3886
Register course number : 3887
Register course number : 3888
Register course number : 3889
Register course number : 3890
Register course number : 3891
Register course number : 3892
Register course number : 3893
Register course number : 3894
Register course number : 3895
Register course number : 3896
Register course number : 3897
Register course number : 3898
Register course number : 3899
Register course number : 3900
Register course number : 3901
Register course number : 3902
Register course number : 3903
Register course number : 3904
Register course number : 3905
Register course number : 3906
Register c

Register course number : 4148
Register course number : 4149
Register course number : 4150
Register course number : 4151
Register course number : 4152
Register course number : 4153
Register course number : 4154
Register course number : 4155
Register course number : 4156
Register course number : 4157
Register course number : 4158
Register course number : 4159
Register course number : 4160
Register course number : 4161
Register course number : 4162
Register course number : 4163
Register course number : 4164
Register course number : 4165
Register course number : 4166
Register course number : 4167
Register course number : 4168
Register course number : 4169
Register course number : 4170
Register course number : 4171
Register course number : 4172
Register course number : 4173
Register course number : 4174
Register course number : 4175
Register course number : 4176
Register course number : 4177
Register course number : 4178
Register course number : 4179
Register course number : 4180
Register c

4204

In [8]:
df1.to_json(orient='values')

NameError: name 'df1' is not defined

In [14]:
indexes = [3851, 3852, 3853, 3854, 3855, 3856, 3857, 3858, 3859, 3860, 3861, 3862, 3863, 3864, 3865, 3866, 3867, 3868, 3869, 3870, 3871, 3872, 3873, 3874, 3875, 3876, 3877, 3878, 3879, 3880, 3881, 3882, 3883, 3884, 3885, 3886, 3887, 3888, 3889, 3890, 3891, 3892, 3893, 3894, 3895, 3896, 3897, 3898, 3899, 3900, 3901, 3902, 3903, 3904, 3905, 3906, 3907, 3908, 3909, 3910, 3911, 3912, 3913, 3914, 3915, 3916, 3917, 3918, 3919, 3920, 3921]

In [23]:
df[['hour', 'longitude', 'latitude', 'elevation','lastStop', 'finalDestination']].iloc[indexes]

,hour,longitude,latitude,elevation,lastStop,finalDestination
3851,07:05:04,47.6382,6.8561,359.0,Strolz,Bois Joli
3852,07:05:23,47.6381,6.8551,359.0,4 As,Bois Joli
3853,07:05:53,47.6381,6.8551,359.0,4 As,Bois Joli
3854,07:06:23,47.6384,6.8517,364.1,4 As,Bois Joli
3855,07:06:39,47.6376,6.8501,375.6,Hatry,Bois Joli
...,...,...,...,...,...,...
3917,07:33:40,47.6342,6.8174,374.4,De Gaulle,Prés D'Aumont
3918,07:33:52,47.6343,6.8192,367.4,Essert,Prés D'Aumont
3919,07:34:22,47.6343,6.8192,367.4,Essert,Prés D'Aumont
3920,07:34:51,47.6342,6.8221,368.7,Essert,Prés D'Aumont


In [26]:
records = df[['hour', 'longitude', 'latitude', 'elevation','lastStop', 'finalDestination']].iloc[indexes].values.tolist()

In [27]:
print(records)

[['07:05:04', 47.6382, 6.8561, 359.0, 'Strolz', 'Bois Joli'], ['07:05:23', 47.6381, 6.8551, 359.0, '4 As', 'Bois Joli'], ['07:05:53', 47.6381, 6.8551, 359.0, '4 As', 'Bois Joli'], ['07:06:23', 47.6384, 6.8517, 364.1, '4 As', 'Bois Joli'], ['07:06:39', 47.6376, 6.8501, 375.6, 'Hatry', 'Bois Joli'], ['07:07:09', 47.6363, 6.8497, 374.7, 'Hatry', 'Bois Joli'], ['07:07:16', 47.6356, 6.8494, 369.5, 'Dubail', 'Bois Joli'], ['07:07:46', 47.6359, 6.8471, 370.4, 'Dubail', 'Bois Joli'], ['07:08:08', 47.6368, 6.8449, 375.0, 'Liberté Follereau', 'Bois Joli'], ['07:08:38', 47.6368, 6.8449, 375.0, 'Liberté Follereau', 'Bois Joli'], ['07:09:08', 47.637, 6.8429, 378.0, 'Liberté Follereau', 'Bois Joli'], ['07:09:29', 47.6367, 6.8407, 374.9, 'Guidon', 'Bois Joli'], ['07:09:59', 47.6367, 6.8404, 374.4, 'Guidon', 'Bois Joli'], ['07:10:29', 47.6363, 6.8374, 372.7, 'Guidon', 'Bois Joli'], ['07:10:49', 47.636, 6.8346, 377.8, 'Gardey', 'Bois Joli'], ['07:11:19', 47.635, 6.831, 375.0, 'Gardey', 'Bois Joli'], ['

In [60]:
df.tail(200)

,ligne,date,hour,longitude,latitude,elevation,lastStop,finalDestination,state,Year
5324722,1,30/10/2020,09:03:04,47.6296,6.8405,378.9,Signoret,Résidences,En course,2020
5324723,1,30/10/2020,09:03:34,47.6285,6.8389,378.4,Signoret,Résidences,En course,2020
5324724,1,30/10/2020,09:04:02,47.6273,6.8356,379.4,La Douce,Résidences,En Terminus,2020
5324725,1,30/10/2020,09:04:32,47.6276,6.8355,379.4,La Douce,Résidences,En course,2020
5324726,1,30/10/2020,09:05:02,47.6276,6.8355,379.4,La Douce,Résidences,En course,2020
...,...,...,...,...,...,...,...,...,...,...
5324917,1,30/10/2020,10:29:19,47.6376,6.8588,359.0,Foch,Valdoie Mairie,En course,2020
5324918,1,30/10/2020,10:29:49,47.6376,6.8588,359.0,Foch,Valdoie Mairie,En course,2020
5324919,1,30/10/2020,10:30:19,47.6381,6.8596,360.0,Foch,Valdoie Mairie,En course,2020
5324920,1,30/10/2020,10:30:41,47.6390,6.8612,360.0,République,Valdoie Mairie,En course,2020
